# 코드 뜯어보기

## 1) Preprocessing

In [2]:
from operator import itemgetter
from scipy.sparse import coo_matrix, save_npz
import numpy as np
from collections import defaultdict

import os
import io
import distutils.dir_util
from collections import Counter
import json
import pickle 

import gc
from scipy.sparse import coo_matrix, lil_matrix, csr_matrix, load_npz
from multiprocessing import Pool
from time import time
from sklearn.svm import LinearSVC

In [3]:
def pickle_dump(data, fname):
    with open(fname, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_load(fname):
    with open(fname, 'rb') as handle:
        data = pickle.load(handle)
    return data

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./" + parent)
    with io.open("./" + fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

def most_popular(playlists, col, topk_count):
    '''가장 많이 등장한 곡/태그를 topk_count만큼 추출하는 함수'''
    c = Counter()
    for doc in playlists:
        c.update(doc[col])
    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

def remove_seen(seen, l):
    '''l 중에서 seen에 이미 있는 아이템은 제거하는 함수'''
    seen = set(seen)
    return [x for x in l if not (x in seen)]

In [4]:
if not(os.path.isdir('data/')):
    os.makedirs('data/')

train = load_json('/data/mountain/train.json')
val = load_json('/data/mountain/val.json')
test = load_json('/data/mountain/test.json')
train = train + val + test

In [4]:
print('rank popular songs/tags...')
data_by_yearmonth = defaultdict(list)
# data_by_yearmonth에 연도별, 년월별로 플레이리스트를 추가  
# data_by_yearmonth = {2013: [플레이리스트1, 플레이리스트2, ...], 2013-07: [플레이리스트1, 플레이리스트2, ...]}
for q in train:
    try:
        data_by_yearmonth[q['updt_date'][0:4]].append(q)
    except:
        pass
    try:
        data_by_yearmonth[q['updt_date'][0:7]].append(q)
    except:
        pass
data_by_yearmonth = dict(data_by_yearmonth)

rank popular songs/tags...


In [ ]:
# 가장 많이 등장한 곡 200개, 태그 50개를 뽑아서 most_popular_results에 저장
# most_popular_results = {'songs': [곡1, 곡2, ...], 'tags': [태그1, 태그2, ...]}
most_popular_results = {}
songs_mp_counter, most_popular_results['songs'] = most_popular(train, "songs", 200)
tags_mp_counter, most_popular_results['tags'] = most_popular(train, "tags", 50)

# 연도별, 년월별로 가장 많이 등장한 곡 200개, 태그 50개를 뽑아서 most_popular_results에 저장 (key)
# most_popular_results = {'songs': [곡1, 곡2, ...], 'tags': [태그1, 태그2, ...], 
#                         'songs2013': [곡1, 곡2, ...], 'tags2013': [태그1, 태그2, ...], 
#                         'songs2013-07': [곡1, 곡2, ...], 'tags2013-07': [태그1, 태그2, ...], ...}
for y in data_by_yearmonth.keys():
    _, most_popular_results['songs' + y] = most_popular(data_by_yearmonth[y], "songs", 200)
    _, most_popular_results['tags' + y] = most_popular(data_by_yearmonth[y], "tags", 50)

In [ ]:
print('split title into words...')
# 태그 빈도수가 5개 이상이고, 태그 길이가 1 이상인 경우 -> all_word_list에 추가
# all_word_list = [('기분전환', 451), ('감성', 429), ...]
all_word_list = []
for t in tags_mp_counter.most_common(): # tags_mp_counter.most_common() = [('기분전환', 451), ('감성', 429), ...]
    if t[1] >= 5 and len(t[0]) > 1: 
        all_word_list.append(t)

In [ ]:
def title_into_words(title):
    '''플레이리스트 제목에 있는 단어가 태그 리스트에 있는 경우, 해당 태그 및 해당 태그와 유사한 태그들을 반환
    e.g. 플리 제목이 '기분전환 여행'이라면 ['기분전환', '기분전환_클럽', '기분전환_편안한', '기분전환_신나는', '여행', '즐거운여행', '선물같은_여행']'''
    word_index_list = []
    # all_word_list에 있는 태그가 플레이리스트 제목에 있는 경우 -> 해당태그, 제목내 인덱스, 빈도수*-1을 word_index_list에 추가 e.g. ['기분전환', 0, -451]
    for word in all_word_list:
        if word[0] in title: # word[0] = '기분전환'
            word_index_list.append([word[0], title.index(word[0]), tags_mp_counter[word[0]] * -1])
    # word_index_list를 '제목내 인덱스','빈도수*-1' 순으로 정렬해서 '태그'만 뽑아서 word_list에 저장
    word_list = [word_index[0] for word_index in sorted(word_index_list, key=itemgetter(1,2))] # word_list = ['기분전환', '감성', ...]
    
    word_list_popular = []
    i=0
    while i < len(word_list):
        # word_list[i]와 유사하게 겹치는 단어가 있는 태그의 경우 same_words에 저장 (e.g. ['기분전환', '기분전환_클럽', '기분전환_편안한', '기분전환_신나는'])
        same_words = [word_list[i]] 
        for j in range(i+1, len(word_list)):
            if word_list[i] in word_list[j] or word_list[j] in word_list[i]:
                same_words.append(word_list[j])
            else:
                break
        i+=len(same_words)
        max_popularity = 0
        word_to_append = None
        # same_words에 있는 word들의 빈도수가 같은 경우, 가장 먼저 나오는 태그를 word_to_append에 저장
        for word in same_words:
            if tags_mp_counter[word] > max_popularity:  
                word_to_append = word
                max_popularity = tags_mp_counter[word]
        # word_to_append가 None이 아닌 경우, word_list_popular에 추가
        if word_to_append is not None:   
            word_list_popular.append(word_to_append)
            
    return word_list_popular

In [ ]:
# train(train+val+test) 그리고 test에 title_words 키 추가
## title_into_words : 플레이리스트 제목에 있는 단어가 태그 리스트에 있는 경우, 해당 태그 및 해당 태그와 유사한 태그들을 반환
for q in train:
    q['title_words'] = title_into_words(q['plylst_title'])
for q in test:
    q['title_words'] = title_into_words(q['plylst_title'])

In [ ]:
# 
print('write train matrix...')
playlist_song_train_matrix = []
p_encode, s_encode, p_decode, s_decode = {}, {}, {}, {}
playlist_idx = 0
song_idx = 0

# songs, tags, title_words에 하나라도 값이 있는 경우 -> 'songs'를 돌면서 플리 인덱스를 p_encode에 저장 & 곡 인덱스를 s_encode에 저장
# p_encode : {플리 id : 플리idx, ...}
# s_encode : {곡 id : 곡idx, ...}
# playlist_song_train_matrix : [[플리idx, 곡idx], ...]
for q in train:
    if len(q['songs']) + len(q['tags']) + len(q['title_words']) >= 1:
        p_encode[q['id']] = playlist_idx
        for s in q['songs']:
            if s not in s_encode.keys():
                s_encode[s] = song_idx
                song_idx += 1
            playlist_song_train_matrix.append([playlist_idx, s_encode[s]])
        playlist_idx += 1
s_decode['@tag_start_idx'] = song_idx  # 태그 인덱스의 시작점

# songs, tags, title_words에 하나라도 값이 있는 경우 -> 'tags' 돌면서 사전에 없으면 곡 인덱스를 s_encode에 저장
# s_encode : {곡 id : 곡idx, 태그 : 태그idx, ...} 
# playlist_song_train_matrix : [[플리idx, 곡idx], [플리idx, 태그idx], ...]
for q in train:
    if len(q['songs']) + len(q['tags']) + len(q['title_words']) >= 1:
        for s in q['tags']:
            if s not in s_encode.keys():
                s_encode[s] = song_idx
                song_idx += 1
            playlist_song_train_matrix.append([p_encode[q['id']], s_encode[s]])
s_decode['@tag_title_start_idx'] = song_idx # title_words 인덱스의 시작점 

# songs, tags, title_words에 하나라도 값이 있는 경우 -> 'title_words' 돌면서 '!title_'+str(제목유사태그)를 s_encode에 저장
# s_encode : {곡 id : 곡idx, 태그 : 태그idx, '!title_제목유사태그' : 제목태그idx ...} 
# playlist_song_train_matrix : [[플리idx, 곡idx], [플리idx, 태그idx], [플리idx, 제목태그idx], ...]
for q in train:
    if len(q['songs']) + len(q['tags']) + len(q['title_words']) >= 1:
        for s in q['title_words']:
            if '!title_' + str(s) not in s_encode.keys():
                s_encode['!title_' + str(s)] = song_idx
                song_idx += 1
            playlist_song_train_matrix.append([p_encode[q['id']], s_encode['!title_' + str(s)]])

In [ ]:
playlist_song_train_matrix.shape[0] # 1150710
playlist_song_train_matrix.shape # (1150710, 2)

In [ ]:
# coo_matrix : sparse matrix (data, (row, col)) 형태로 저장 (shape: 행렬의 크기)
# playlist_song_train_matrix : [[플리idx, 곡idx], [플리idx, 태그idx], [플리idx, 제목태그idx], ...]
playlist_song_train_matrix = np.array(playlist_song_train_matrix)
playlist_song_train_matrix = coo_matrix((np.ones(playlist_song_train_matrix.shape[0]),  # data (shape: (1150710,))
                                         (playlist_song_train_matrix[:,0],              # row : 플리idx
                                          playlist_song_train_matrix[:,1])),            # col : 곡/태그/제목태그idx
                                        shape=(playlist_idx,song_idx))                  # shape : (플리id 개수, 곡/태그/제목태그 개수)
# playlist_song_train_matrix 원소 : 1 (플리id에 곡/태그/제목태그가 포함되어 있으면 1, 아니면 0)

# [플리idx, 곡/태그/제목태그idx] sparse matrix를 npz 파일로 저장
save_npz('data/playlist_song_train_matrix.npz', playlist_song_train_matrix)

In [ ]:
# s_decode : {곡 idx : 곡 id, 태그 idx : 태그, '!title_제목유사태그' : 제목태그 ...}
for s in s_encode.keys():
    s_decode[s_encode[s]] = s

# p_encode, s_decode를 pickle 파일로 저장
# p_encode : {플리 id : 플리idx, ...}
pickle_dump(s_decode, 'data/song_label_decoder.pickle')
pickle_dump(p_encode, 'data/playlist_label_encoder.pickle')

In [ ]:
# train+val+test 플리별로 가장 많이 등장한 제목유사태그 50개 뽑아서 title_words_mp_counter에 저장 (key)
title_words_mp_counter, _ = most_popular(train, "title_words", 50)

In [ ]:
print('write test item indices...')
for q in test:
    # songs, tags, title_words에 하나라도 값이 있는 경우 
    if len(q['songs']) + len(q['tags']) + len(q['title_words']) >= 1:
        # 가장 많이 등장한 곡 200개의 빈도수 평균 + 가장 많이 등장한 태그 50개의 빈도수 평균 + 가장 많이 등장한 제목유사태그 50개의 빈도수 평균이 1을 초과하는 경우
        if np.mean([songs_mp_counter[i] for i in q['songs']] \
                    + [tags_mp_counter[i] for i in q['tags']] \
                    + [title_words_mp_counter[i] for i in q['title_words']]) > 1:
            items = [s_encode[s] for s in q['songs'] + q['tags']]  # items : [곡idx, 곡idx, ..., 태그idx, 태그idx, ...]
            try:
                for s in q['title_words']:
                    if '!title_' + str(s) in s_encode.keys(): # 제목유사태그가 s_encode에 있는 경우
                        items.append(s_encode['!title_' + str(s)]) # items : [곡idx, 곡idx, ..., 태그idx, 태그idx, ..., 제목태그idx, 제목태그idx, ...]
            except KeyError: 
                q['title_words'] = [] # s_encode에에 없으면 title_words 키의 값을 빈 리스트로 변경
            # test 데이터셋에 items 키 추가
            q['items'] = items
    
    # 테스트셋 플리 년월에 해당하는 'songsYYYY-MM'이 most_popular_results에 있는 경우
    if 'songs'+q['updt_date'][0:7] in most_popular_results.keys():
        # 테스트셋에 'songs_mp' 키 추가 <- {(songsYYYY-MM에 있는 곡 | 가장 인기있는 곡) - 테스트셋 플리에 있는 곡} 100개
        q['songs_mp'] = (remove_seen(q['songs'],
                                     most_popular_results['songs'+q['updt_date'][0:7]] + remove_seen(most_popular_results['songs'+q['updt_date'][0:7]], 
                                                                                                     most_popular_results['songs']\
                                                                                                    )\
                                    )\
                        )[:100]
        # 테스트셋에 'tags_mp' 키 추가 <- {(tagsYYYY-MM에 있는 태그 | 가장 인기있는 태그) - 테스트셋 플리에 있는 태그} 10개
        q['tags_mp'] = (remove_seen(q['tags'],
                                    most_popular_results['tags'+q['updt_date'][0:7]] + remove_seen(most_popular_results['tags'+q['updt_date'][0:7]],
                                                                                                   most_popular_results['tags']\
                                                                                                  )\
                                   )\
                       )[:10]
    
    # 테스트셋 플리 연도에 해당하는 'songsYYYY'이 most_popular_results에 있는 경우
    elif 'songs'+q['updt_date'][0:4] in most_popular_results.keys():
        # 테스트셋에 'songs_mp' 키 추가 <- {(songsYYYY에 있는 곡 | 가장 인기있는 곡) - 테스트셋 플리에 있는 곡} 100개
        q['songs_mp'] = (remove_seen(q['songs'],most_popular_results['songs'+q['updt_date'][0:4]] + remove_seen(most_popular_results['songs'+q['updt_date'][0:4]], most_popular_results['songs'])))[:100]
        # 테스트셋에 'tags_mp' 키 추가 <- {(tagsYYYY에 있는 태그 | 가장 인기있는 태그) - 테스트셋 플리에 있는 태그} 10개
        q['tags_mp'] = (remove_seen(q['tags'],most_popular_results['tags'+q['updt_date'][0:4]] + remove_seen(most_popular_results['tags'+q['updt_date'][0:4]], most_popular_results['tags'])))[:10]
    
    # 테스트셋 플리 연도 및 연월에 해당하는 'songsYYYY-MM'과 'songsYYYY 둘 다 most_popular_results에 없는 경우
    else:
        # 테스트셋에 'songs_mp' 키 추가 <- {가장 인기있는 곡 - 테스트셋 플리에 있는 곡} 100개
        q['songs_mp'] = remove_seen(q['songs'], most_popular_results['songs'][:100])
        # 테스트셋에 'tags_mp' 키 추가 <- {가장 인기있는 태그 - 테스트셋 플리에 있는 태그} 10개
        q['tags_mp'] = remove_seen(q['tags'],most_popular_results['tags'][:10])

In [ ]:
# 'items', 'songs_mp', 'tags_mp' 키를 추가한 test 데이터셋을 json 파일로 저장
write_json(test, 'data/test_items.json')

## 2) Inference

In [ ]:
# preprocessing 불러오기
s_decode = pickle_load('data/song_label_decoder.pickle')
p_encode = pickle_load('data/playlist_label_encoder.pickle')
tag_start_idx = s_decode['@tag_start_idx']
tag_title_start_idx = s_decode['@tag_title_start_idx']

In [ ]:
# playlist_song_train_matrix : [플리idx, 곡/태그/제목태그idx] sparse matrix
print("load train matrix...")
playlist_song_train_matrix = load_npz('data/playlist_song_train_matrix.npz')  # shape : (플리id 개수, 곡/태그/제목태그 개수)

# song_playlist_train_matrix_raw : [곡/태그/제목태그idx, 플리idx] sparse matrix
song_playlist_train_matrix_raw = lil_matrix(playlist_song_train_matrix.transpose()) # shape : (곡/태그/제목태그 개수, 플리id 개수)

In [ ]:
gc.collect()  # 메모리 정리 (불필요하게 쌓인 메모리를 해제)

In [ ]:
song_playlist_train_matrix_raw.sum(axis=1) # 행별 합계 # shape : (곡/태그/제목태그 개수, 1)
song_playlist_train_matrix_raw.sum(axis=0) # 열별 합계 # shape : (1, 플리id 개수)

In [ ]:
I_song = np.array(song_playlist_train_matrix_raw.sum(axis=1)).flatten() # shape : (곡/태그/제목태그 개수, )
I_list = np.array(song_playlist_train_matrix_raw.sum(axis=0)).flatten() # shape : (플리id 개수, )

In [ ]:
# 테스트셋 불러오기
print("load test data...")
test = load_json('data/test_items.json')

# 테스트셋에 items 키가 있는 경우
# test_item_indices = {플리id1: [곡idx, 곡idx, ..., 태그idx, 태그idx, ..., 제목태그idx, 제목태그idx, ...], ...}
# test_playlist_id = [플리id1, 플리id2, ...]
test_item_indices = dict()
test_playlist_id = []
for q in test:
    if 'items' in q.keys():
        test_item_indices[q['id']] = q['items']
        test_playlist_id.append(q['id'])

In [ ]:
def neighbor_based_cf(playlist_id):
    '''
    전역 변수 : 
    test_item_indices = {플리id1: [곡idx, 곡idx, ..., 태그idx, 태그idx, ..., 제목태그idx, 제목태그idx, ...], ...}
    song_playlist_train_matrix_raw : shape (곡/태그/제목태그 개수, 플리id 개수) # shape: (615142, 115071)
    '''
    item_indices = test_item_indices[playlist_id] # 현재 플리id에 해당하는 [곡idx, 곡idx, ..., 태그idx, 태그idx, ..., 제목태그idx, 제목태그idx, ...]

    alpha, beta, theta = 0.9, 0.7, 0.99
    Cr = 0.4 + (100 - np.shape(item_indices)[0]) * 0.0055  # np.shape(item_indices)[0] : 플리에 포함된 곡/태그/제목태그 개수
    # Cr : 0.2 ~ 1 사이의 값
    # Cr : 플리에 포함된 곡/태그/제목태그 개수가 100개일 때 0.4, 1개일 때 0.4 + (100-1)*0.0055 = 0.9445
    if Cr < 0.2:
        Cr = 0.2
    elif Cr > 1:
        Cr = 1

    song_playlist_train_matrix = lil_matrix(song_playlist_train_matrix_raw) # shape (곡/태그/제목태그 개수, 플리id 개수)
    song_playlist_train_matrix[:,p_encode[playlist_id]] = 0  # 해당 플리에 해당하는 모든 행을 0으로 설정 >> 현재 플리와 연관된 곡/태그/제목태그들을 제거하는 역할

    weight = song_playlist_train_matrix[item_indices, :].multiply(np.power(1e-1 + I_list, beta - 1)).sum(axis=0) 
    weight = np.array(weight).flatten() 
    weight = np.power(weight,theta) 
    # >> weight: (플리id 개수, ) : 현재 플리와 연관된 곡/태그/제목태그들의 플리별 가중치
    
    value = song_playlist_train_matrix[item_indices, :].multiply(weight) # shape : (플리id에 해당하는 곡/태그/제목태그 개수, 플리 개수) : 현재 플리 value에 가중치 곱
    value = value.dot(song_playlist_train_matrix.transpose()) # shape : (플리id에 해당하는 곡/태그/제목태그 개수, 곡/태그/제목태그 개수)
    I_song_i = np.power(1e-1+I_song[item_indices], -alpha) # shape : (플리id에 해당하는 곡/태그/제목태그 개수, ) 
    value = value.multiply(I_song_i.reshape((-1,1)))  
    value = value.multiply(np.power(1e-1+I_song,alpha-1)) 
    value = csr_matrix(value) # csr_matrix로 변환
    # >> value: (플리id에 해당하는 곡/태그/제목태그 개수, 곡/태그/제목태그 개수) 

    predictions = lil_matrix(value) # value를 lil_matrix로 변환
    label = np.zeros(song_playlist_train_matrix.shape[0]) # 곡/태그/제목태그 개수만큼 0으로 채워진 배열
    label[item_indices] = 1  # 해당 플리id 곡/태그/제목태그 위치에 1을 넣음
    # >> label: (곡/태그/제목태그 개수, )
    
    clf = LinearSVC(C=Cr, class_weight={0:1,1:1}, tol=1e-6, dual = True, max_iter=360000) # C: Regularization parameter 
    clf.fit(predictions.transpose(),label)
    predictions = clf.decision_function(predictions.transpose()) # proba
    predictions = np.argsort(np.array(predictions).flatten() - min(predictions))[::-1] # 내림차순 정렬된 인덱스 반환

    # 출력 : 곡 400개 인덱스 + 태그 100개 인덱스 
    return np.array(list(predictions[predictions < tag_start_idx][:400]) + list(predictions[(predictions >= tag_start_idx) & (predictions < tag_title_start_idx)][:100]))  

In [ ]:
# 예측 시작
print("predictions begin...")
pool = Pool(23)  # 멀티프로세싱을 위한 Pool 객체 생성
results = pool.map(neighbor_based_cf, test_playlist_id) # test_playlist_id를 인자로 받아 neighbor_based_cf 함수를 병렬로 실행한 결과를 results에 저장
pool.close() # 멀티프로세싱을 위한 Pool 객체 종료
pool.join() # 병렬로 실행한 프로세스가 종료될 때까지 대기

In [ ]:
# 예측 결과 딕셔너리 생성
# test_playlist_id = [플리id1, 플리id2, ...]
# prediction_results : {플리id1: {"songs": [곡id1, 곡id2, ..., 곡id400], "tags": [태그1, 태그2, ..., 태그400]}, ...}
prediction_results = {}
for i in range(len(results)):
    prediction_results[test_playlist_id[i]] = {"songs": [s_decode[s] for s in results[i][:400]], 
                                               "tags": [s_decode[s] for s in results[i][400:]]}

In [ ]:
# 예측 결과 저장
# 테스트셋의 플리id가 예측 결과에 있는 경우 -> (예측 결과에 있는 곡/태그 - 테스트셋 플리에 있는 곡/태그) 곡 100개, 태그 10개를 answers에 추가
# 테스트셋의 플리id가 예측 결과에 없는 경우 -> (가장 인기있는 곡/태그 - 테스트셋 플리에 있는 곡/태그) 곡 100개, 태그 10개를 answers에 추가
print("write results.json...")
answers = []
for q in test:
    if q['id'] in test_playlist_id:
        answers.append({'id': q['id'],
        'songs': remove_seen(q['songs'], prediction_results[q['id']]['songs'])[:100],
        'tags': remove_seen(q['tags'], prediction_results[q['id']]['tags'])[:10] })
    else:
        answers.append({'id': q['id'],
        'songs': remove_seen(q['songs'], q['songs_mp'])[:100],
        'tags': remove_seen(q['tags'], q['tags_mp'])[:10] })
    # 가장 최근에 추가한 플리의 곡/태그가 100개/10개가 안되는 경우 -> (answers에 있는 곡/태그 + (가장 인기있는 곡/태그 - 플리에 있는 곡/태그)) 곡 100개, 태그 10개로 answers 대체
    if len(answers[len(answers)-1]['songs']) < 100 or len(answers[len(answers)-1]['tags']) < 10:
        answers[len(answers)-1]['songs'] = (answers[len(answers)-1]['songs'] + remove_seen(q['songs'] + answers[len(answers)-1]['songs'], q['songs_mp']))[:100]
        answers[len(answers)-1]['tags'] = (answers[len(answers)-1]['tags'] + remove_seen(q['tags'] + answers[len(answers)-1]['tags'], q['tags_mp']))[:10]

# answers를 json 파일로 저장
write_json(answers, 'results.json')
print('end')

In [20]:
path = '/data/mountain/'
# 피클파일 불러오기
import pickle
   
with open(path + 'mel_mean_scaled.pkl', 'rb') as f:
    mel_mean_scaled = pickle.load(f)
mel_mean_scaled

{536701: 0.89586484,
 536868: 0.8183825,
 536242: 0.7814734,
 536932: 0.8514207,
 536040: 0.79613787,
 536363: 0.9010694,
 536798: 0.61883736,
 536900: 0.7075604,
 536328: 0.8811539,
 536750: 0.7516645,
 536676: 0.8173888,
 536702: 0.8694962,
 536792: 0.71946573,
 536650: 0.87598616,
 536399: 0.865579,
 536777: 0.78332543,
 536094: 0.8027653,
 536991: 0.6797637,
 536999: 0.79618645,
 536810: 0.84787756,
 536302: 0.88237774,
 536321: 0.87403667,
 536566: 0.80546224,
 536947: 0.58290553,
 536789: 0.876712,
 536414: 0.85414726,
 536640: 0.93262124,
 536797: 0.80365413,
 536982: 0.87946904,
 536907: 0.7917751,
 536811: 0.86502326,
 536593: 0.82682645,
 536850: 0.8601216,
 536995: 0.79001725,
 536985: 0.82399094,
 536014: 0.7929543,
 536017: 0.8800326,
 536216: 0.8161643,
 536604: 0.7624959,
 536372: 0.7680458,
 536083: 0.7789482,
 536535: 0.8653835,
 536068: 0.7496331,
 536244: 0.82028234,
 536098: 0.5446862,
 536197: 0.7491325,
 536574: 0.80481064,
 536556: 0.58011854,
 536087: 0.89657044

In [26]:
# mel_mean_scaled 딕셔너리를 데이터프레임으로 변환
melmean = pd.DataFrame.from_dict(mel_mean_scaled, orient='index', columns=['mel_mean_scaled'])
melmean.reset_index(inplace=True)
melmean.head()

,index,mel_mean_scaled
0,536701,0.895865
1,536868,0.818383
2,536242,0.781473
3,536932,0.851421
4,536040,0.796138


In [27]:
# train 불러오기
train = pd.read_json(path + 'train.json')
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [33]:
plylst_song_mean = train[['id', 'songs']].explode('songs')
# songs에 맞춰서 melmean을 합침
plylst_song_mean['mean'] = plylst_song_mean['songs'].map(melmean.set_index('index')['mel_mean_scaled'])
plylst_song_mean.head()

,id,songs,mean
0,61281,525514,0.766784
0,61281,129701,0.923905
0,61281,383374,0.875784
0,61281,562083,0.771357
0,61281,297861,0.835334


In [ ]:
import numpy as np
import pandas as pd
import datetime
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

# train test 분리
TRAIN_SIZE = 0.75
melmean = shuffle(melmean, random_state=12)
cutoff = int(TRAIN_SIZE * len(melmean))
ratings_train = melmean.iloc[:cutoff]
ratings_test = melmean.iloc[cutoff:]

# Sparse matrix 사용
data = np.array(melmean['mel_mean_scaled'])
row_indices = np.array(melmean['index']) # plylst_id
col_indices = np.array(melmean['index'])
melmean = csr_matrix((data, (row_indices, col_indices)), dtype=int)

In [ ]:
# Created or modified on Nov 2022
# Author: 임일
# Sparse matrix 3

import numpy as np
import pandas as pd
import datetime
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

# 데이터 불러오기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
#ratings = pd.read_csv('C:/RecoSys/Data/u.data', names=r_cols,  sep='\t',encoding='latin-1')         # 100K data
#ratings = pd.read_csv('C:/RecoSys/Data/ratings-1m.csv', names=r_cols,  sep=',',encoding='latin-1')  # 1M data
ratings = pd.read_csv('Week 13 - Codes_data/ratings-20m.csv', names=r_cols,  sep=',',encoding='latin-1') # 20M data
ratings = ratings.drop('timestamp', axis=1)

# train test 분리
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=12)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

# Sparse matrix 사용
data = np.array(ratings['rating'])
row_indices = np.array(ratings['user_id'])
col_indices = np.array(ratings['movie_id'])
ratings = csr_matrix((data, (row_indices, col_indices)), dtype=int)

